In [29]:

# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import kaleido
# Hugging Face Transformers and datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from datasets import load_dataset, Dataset
# scikit-learn for dataset splitting
from sklearn.model_selection import train_test_split


this first cell is for loading my dataset, splitting it into a test and training set. then saving the respective datasets to csv files

In [30]:
def IMPORT_BABEv3():
    dataset = load_dataset("mediabiasgroup/BABE-v3")
    df = pd.DataFrame(dataset["train"])
    DF_TRAIN, DF_TEST = train_test_split(df, test_size=0.20, random_state=42)
    DF_TRAIN.to_csv("TRAINING_DATAFRAME.csv", index=False)
    DF_TEST.to_csv("TESTING_DATAFRAME.csv", index=False)

IMPORT_BABEv3()

Using the latest cached version of the dataset since mediabiasgroup/BABE-v3 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/oscar/.cache/huggingface/datasets/mediabiasgroup___BABE-v3/default/0.0.0/5493fe1686f29d4fce6912ccf9e2e03780493bd6 (last modified on Mon Mar 18 10:23:39 2024).


This is for running the pre trained models

In [31]:
#This is for the analysis of the D1V1DE bias-detection model (https://huggingface.co/D1V1DE/bias-detection?text=I+like+you.+I+love+you)
def PRE_TEST_D1V1DE():
    try:
        pipe = pipeline("text-classification", model="D1V1DE/bias-detection")
        CurrentDF = pd.read_csv("TESTING_DATAFRAME.csv")
        CurrentDF['Predicted'] = 'XXX'
        CurrentDF.drop(['news_link','outlet','label','label_opinion','biased_words'], axis=1, inplace=True)
        for index, row in CurrentDF.iterrows():
            text_data = row['text']
            bias = pipe(text_data)
            CurrentDF.at[index, 'Predicted'] = bias[0]["label"]
        CurrentDF.to_csv("temp.csv", index = False)
        
        Type =  GEN_EVAL_ScorePerTopic(CurrentDF, "type")
        Topic = GEN_EVAL_ScorePerTopic(CurrentDF, "topic")
        results = Type.join(Topic).T
        results['score'] = results['score'] * 100
        results.to_csv("PRED1V1DE.csv", index=True)
        print("D1V1DE/bias-detection examination completed successfully")
        return results
    except Exception as e:
        print("D1V1DE/bias-detection failed")
        print(e)

PRE_TEST_D1V1DE()

D1V1DE/bias-detection examination completed successfully


,correct,count,score,field
NaN,133.0,289.0,46.020761,type
left,64.0,203.0,31.527094,type
center,122.0,136.0,89.705882,type
right,66.0,197.0,33.502538,type
white-nationalism,6.0,30.0,20.0,topic
gender,13.0,22.0,59.090909,topic
#metoo,6.0,7.0,85.714286,topic
marriage-equality,20.0,68.0,29.411765,topic
taxes,29.0,54.0,53.703704,topic
student-debt,7.0,22.0,31.818182,topic


In [39]:
#This is for the analysis of the finetuned D1V1DE bias-detection model (https://huggingface.co/D1V1DE/bias-detection?text=I+like+you.+I+love+you)
def FIN_TEST_D1V1DE():
    try:
        # Load the fine-tuned model as a pipeline for text classification
        pipe = pipeline("text-classification", model="./fine_tuned/D1V1DE")
        # Load the test dataset
        test_df = pd.read_csv("TESTING_DATAFRAME.csv")
        test_df['Predicted'] = 'XXX'  # Initialize the Predicted column
        # Drop unnecessary columns
        test_df.drop(['news_link', 'outlet', 'label', 'label_opinion', 'biased_words'], axis=1, inplace=True)
        # Make predictions and store in the DataFrame
        for index, row in test_df.iterrows():
            text_data = row['text']
            prediction = pipe(text_data)
            test_df.at[index, 'Predicted'] = prediction[0]["label"]

        # Save the DataFrame with predictions to a temporary CSV file
        test_df.to_csv("temp_predictions.csv", index=False)

        # Analyze results by type and topic
        Type = GEN_EVAL_ScorePerTopic(test_df, "type")
        Topic = GEN_EVAL_ScorePerTopic(test_df, "topic")

        # Combine results and calculate scores
        results = Type.join(Topic).T
        results['score'] = results['score'] * 100

        # Save final results to a CSV file
        results.to_csv("FIN_TEST_D1V1DE_RESULTS.csv", index=True)
        print("Fine-tuned D1V1DE/bias-detection model evaluation completed successfully")
        return results
    except Exception as e:
        print("Evaluation of the fine-tuned D1V1DE/bias-detection model failed")
        print(e)

# Example usage
FIN_TEST_D1V1DE()

Fine-tuned D1V1DE/bias-detection model evaluation completed successfully


,correct,count,score,field
NaN,119.0,289.0,41.176471,type
left,85.0,203.0,41.871921,type
center,128.0,136.0,94.117647,type
right,83.0,197.0,42.13198,type
white-nationalism,8.0,30.0,26.666667,topic
gender,14.0,22.0,63.636364,topic
#metoo,7.0,7.0,100.0,topic
marriage-equality,22.0,68.0,32.352941,topic
taxes,30.0,54.0,55.555556,topic
student-debt,12.0,22.0,54.545455,topic


the bellow section is for fine tuning models and their evaluations

In [33]:
#fine tuning the model with my training dataset
def FIN_TRAIN_D1V1DE(training_data_path):
    # Load and preprocess the dataset
    df = pd.read_csv(training_data_path)
    df = df[['text', 'label']]
    df_train, df_val = train_test_split(df, test_size=0.1)
    train_dataset = Dataset.from_pandas(df_train)
    val_dataset = Dataset.from_pandas(df_val)
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained("D1V1DE/bias-detection")
    # Tokenization function
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    # Tokenize the datasets
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
    # Load the pretrained model
    model = AutoModelForSequenceClassification.from_pretrained("D1V1DE/bias-detection")
    # Training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10
    )
    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset
    )
    # Fine-tune the model
    trainer.train()
    # Save the fine-tuned model
    tokenizer.save_pretrained("./fine_tuned/D1V1DE")
    model.save_pretrained("./fine_tuned/D1V1DE")


#code for fine-tuning individual models
#FIN_TRAIN_D1V1DE("TRAINING_DATAFRAME.csv")

Map: 100%|██████████| 330/330 [00:00<00:00, 10420.63 examples/s]
/Users/oscar/miniforge3/envs/oscar/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  2%|▏         | 10/558 [00:11<10:27,  1.14s/it]

{'loss': 0.3819, 'grad_norm': 24.358970642089844, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}


  4%|▎         | 20/558 [00:23<10:12,  1.14s/it]

{'loss': 0.3204, 'grad_norm': 23.58262825012207, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.11}


  5%|▌         | 30/558 [00:34<09:58,  1.13s/it]

...

{'loss': 0.2425, 'grad_norm': 23.04095458984375, 'learning_rate': 1.5517241379310346e-05, 'epoch': 2.9}


 99%|█████████▊| 550/558 [11:29<00:10,  1.32s/it]

{'loss': 0.2372, 'grad_norm': 29.879043579101562, 'learning_rate': 6.896551724137932e-06, 'epoch': 2.96}


100%|██████████| 558/558 [11:38<00:00,  1.25s/it]


{'train_runtime': 698.96, 'train_samples_per_second': 12.73, 'train_steps_per_second': 0.798, 'train_loss': 0.33937042673856127, 'epoch': 3.0}


Bellow is all nececary code for evaluations

In [34]:
#used for doing all my scoring and accuracy testing of models using the bias dataset
#variables need renaming and potentailly this code could be sped up
def GEN_EVAL_ScorePerTopic(data, field):
    unique_items = data[field].unique().tolist()
    scores = {item: {"correct": 0, "count": 0} for item in unique_items}
    for index, row in data.iterrows():
        correct = False
        if pd.isna(row[field]) and row['Predicted'] == 'NEUTRAL':
            correct = True
        elif not pd.isna(row[field]) and row['Predicted'] != 'NEUTRAL':
            correct = True
        scores[row[field]]['count'] += 1
        if correct:
            scores[row[field]]['correct'] += 1
    current = {}
    for item in scores:
        scores[item]['score'] = scores[item]['correct'] / scores[item]['count']
    scores = pd.DataFrame(scores).T
    scores["field"] = field #could do with changing from field
    return pd.DataFrame(scores).T

In [35]:
def GEN_EVAL_evaluate_model(model, tokenizer, eval_dataset):
    #used for evaluating performance of models in terms of speed. needs further research
    # Initialize the trainer
    trainer = Trainer(model=model)
    # Tokenize the evaluation dataset
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)
    tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
    # Evaluate the model
    results = trainer.evaluate(tokenized_eval_dataset)
    return results

using the standard python tools for evaluating models performance

In [36]:
def EVAL_D1V1DE_evaluate_model():
    #evalating fine tuned D1V1DE model
    tokenizer = AutoTokenizer.from_pretrained("D1V1DE/bias-detection")
    model = AutoModelForSequenceClassification.from_pretrained("fine_tuned/D1V1DE")
    df_eval = pd.read_csv('TESTING_DATAFRAME.csv')
    eval_dataset = Dataset.from_pandas(df_eval[['text', 'label']])
    # Evaluate the model
    evaluation_results = GEN_EVAL_evaluate_model(model, tokenizer, eval_dataset)

    print("FIND1V1DE: ", evaluation_results)


    # Replace 'D1V1DE/original-model-name' with the correct path to the original model on Hugging Face
    original_model = AutoModelForSequenceClassification.from_pretrained("D1V1DE/bias-detection")
    # Evaluate the original model
    original_evaluation_results = GEN_EVAL_evaluate_model(original_model, tokenizer, eval_dataset)

    # Print the evaluation results of the original model
    print("PRED1V1DE: ", original_evaluation_results)

EVAL_D1V1DE_evaluate_model()

/Users/oscar/miniforge3/envs/oscar/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
100%|██████████| 104/104 [00:16<00:00,  6.30it/s]


FIND1V1DE:  {'eval_loss': 0.4441041946411133, 'eval_runtime': 16.5461, 'eval_samples_per_second': 49.861, 'eval_steps_per_second': 6.285}


100%|██████████| 104/104 [00:16<00:00,  6.31it/s]

PRED1V1DE:  {'eval_loss': 0.29630759358406067, 'eval_runtime': 16.5132, 'eval_samples_per_second': 49.96, 'eval_steps_per_second': 6.298}


bellow is code for visualisations and evaluation assistive tools

In [37]:
def GEN_VIS_star(data,key):
    FilteredData = data[data['field'] == key]
    # Preparing the data for the star plot (radar chart)
    FilteredData = FilteredData.reset_index()
    FilteredData['index'] = FilteredData['index'].fillna('No Bias')
    labels=FilteredData['index']
    #print(labels)
    stats=FilteredData['score']
    #print(stats)

    # Create radar chart
    angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
    stats=np.concatenate((stats,[stats[0]]))
    angles+=angles[:1]
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, stats, color='blue', alpha=0.25)
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels)

    # Display the plot
    plt.title('Star Plot of '+  str(key) +' vs. accuracy rating')
    plt.show()